In [1]:
%run __init__.ipynb

Successfully connected to MongoDB


In [2]:
#--- Covid-19 France data updated everyday at 19:00PM
%run "covid-19_france.ipynb"

Successfully connected to MongoDB
FRANCE_INIT successfully saved in FRANCE_INIT
Dataframe FRANCE_INIT successfully save in database covid-19 in MongoDB. Time: --- 3.6820900440216064 secnds ---
CPU times: user 1.67 s, sys: 35.6 ms, total: 1.71 s
Wall time: 4.41 s
FRANCE_DB_CONCAT successfully saved in FRANCE_DB_CONCAT
Dataframe FRANCE_DB_CONCAT successfully save in database covid-19 in MongoDB. Time: --- 131.8257954120636 secnds ---
CPU times: user 25.3 s, sys: 2.42 s, total: 27.7 s
Wall time: 2min 12s
FRANCE_DB_CONSO successfully saved in FRANCE_DB_CONSO
Dataframe FRANCE_DB_CONSO successfully save in database covid-19 in MongoDB. Time: --- 205.06097149848938 secnds ---
CPU times: user 33.5 s, sys: 2.82 s, total: 36.3 s
Wall time: 3min 27s
FRANCE_DB_ALL successfully saved in FRANCE_DB_ALL
Dataframe FRANCE_DB_ALL successfully save in database covid-19 in MongoDB. Time: --- 110.14483618736267 secnds ---
CPU times: user 23.5 s, sys: 1.45 s, total: 24.9 s
Wall time: 1min 59s
FRANCE_DB_TREND

In [3]:
%%time

#--- Load data source
db_all_fr = get_datasource('FRANCE_DB_ALL')
db_trend_fr = get_datasource('FRANCE_DB_TREND')

CPU times: user 588 ms, sys: 16.8 ms, total: 604 ms
Wall time: 1.49 s


In [4]:
%%time

#-- Covid-19 - DATA.GOUV.FR TENDANCES
def domain_105(df,domain_num):
    #-- Create Datamodel
    domain = df.copy()
    domain = pd.DataFrame({
        'ENTITY': 'France',
        'SCENARIO': domain['SCENARIO'],
        'GROUPS': domain['STATUS_NAME'],
        'GROUPS_ORDER': domain['STATUS_ORDER'],
        'DATE_ORDER': domain['DATE_ORDER'],
        'BOTTOM_FILTER': 'Depuis le début',
        'UPPER_FILTER_M': domain['METRIC'],
        'UPPER_FILTER_R': domain['LABEL'],
        'LABEL_GROUPS': domain['LABEL_GROUPS'],
        'VALUE': domain['VALUE'],
        'UNIT_VALUE': domain['UNIT'],
        'PRECISION': domain['PRECISION'],
    })
    
    #-- Rename upper filter
    domain.loc[domain['UPPER_FILTER_M'] == 'VALUE','UPPER_FILTER_M'] = 'EVOLUTION'
    domain.loc[domain['UPPER_FILTER_M'] == 'VARV','UPPER_FILTER_M'] = 'VARIATION EN NB'
    domain.loc[domain['UPPER_FILTER_M'] == 'VARP','UPPER_FILTER_M'] = 'VARIATION EN %'
    
    #-- Create new date scenario in BOTTOM_FILTER
    df1 = get_lastdays(domain, 15,"15 derniers jours",'BOTTOM_FILTER')
    domain = pd.concat([domain,df1],axis=0)
    
    #-- Filter upperfilter value to get France and detailed by region
    domain_dep = domain[domain['LABEL_GROUPS'] != 'France']
    domain_fra = domain[domain['UPPER_FILTER_R'] == 'France']
    domain = pd.concat([domain_dep,domain_fra],axis=0)
    
    #-- Reset indexes & save domain
    domain = domain.reset_index(drop=True)
    domain['LAST_UPDATE']= datetime.now().strftime('%d/%m/%Y %H:%M:%S')
    df_save(domain,domain_num,'csv')
    if USE_MONGO:
        naas_drivers.mongo.send(domain,domain_num,DB_APP,True)
    
    #-- Create upper filter domain
    upper_filter = domain[['UPPER_FILTER_R']].drop_duplicates().sort_values(by='UPPER_FILTER_R', ascending=True).reset_index(drop=True)
    upper_filter['ORDER'] = upper_filter.index + 1
    upper_filter.loc[upper_filter['UPPER_FILTER_R'] == 'France', 'ORDER'] = 0
    upper_filter = upper_filter.sort_values(by='ORDER',ascending=True)
    df_save(upper_filter,'105_UPPER_FILTER','csv')
    if USE_MONGO:
        naas_drivers.mongo.send(upper_filter,'105_UPPER_FILTER',DB_APP,True)
    return domain

domain105 = domain_105(db_trend_fr,'105')
# domain105

105 successfully saved in 105
Dataframe 105 successfully save in database app-wsr in MongoDB. Time: --- 56.93586564064026 secnds ---
105_UPPER_FILTER successfully saved in 105_UPPER_FILTER
Dataframe 105_UPPER_FILTER successfully save in database app-wsr in MongoDB. Time: --- 0.03388214111328125 secnds ---
CPU times: user 21.1 s, sys: 1.11 s, total: 22.2 s
Wall time: 1min 1s


In [5]:
%%time

#-- Covid-19 - DATA.GOUV.FR CLASSEMENT
def domain_107(df,domain_num):
    domain = df.copy()
    domain = pd.DataFrame({
        'ENTITY': 'France',
        'SCENARIO': domain['SCENARIO'],
        'DATE_ORDER': domain['DATE_ORDER'],
        'CHILD': domain['LABEL'],
        'PARENT': domain['LABEL_GROUPS'],
        'UPPER_FILTER_M': domain['STATUS_NAME'],
        'ORDER':domain['STATUS_ORDER'],
        'VALUE': domain['VALUE'],
        'VAR': domain['VARV'],
        'VARP': domain['VARP'],
        'UNIT_VARP': ' %',
        'PRECISION_VALUE': ',.0f',
        'PRECISION_VAR': '+,.0f',
        'PRECISION_VARP': '+,.2f',
    })
    
    domain.loc[domain['PARENT'] == 'France', 'PARENT'] = 'root'
    
    #-- Reset indexes & save domain
    domain = domain.reset_index(drop=True)
    domain['LAST_UPDATE']= datetime.now().strftime('%d/%m/%Y %H:%M:%S')
    df_save(domain,domain_num,'csv')
    if USE_MONGO:
        naas_drivers.mongo.send(domain,domain_num,DB_APP,True)
    return domain

domain107 = domain_107(db_all_fr,'107')
# domain107

107 successfully saved in 107
Dataframe 107 successfully save in database app-wsr in MongoDB. Time: --- 43.929227352142334 secnds ---
CPU times: user 12 s, sys: 768 ms, total: 12.7 s
Wall time: 44.8 s


In [7]:
%run "map-chart_france.ipynb"

Successfully connected to MongoDB
Data already updated !


UsageError: Line magic function `%stop` not found.


107_FRANCE successfully saved in 107_FRANCE
CPU times: user 224 ms, sys: 1.11 ms, total: 225 ms
Wall time: 735 ms
VALUE : Min: 0, Max: 568,Average: 48.0, AvrMax: 0.08,Med: 10.0, MedMax: 0.02


AttributeError: module 'bob' has no attribute 'makeStaticGettable'

AttributeError: module 'bob' has no attribute 'makeStaticGettable'